# Capstone 2

## Polish Bankruptcy

## Pre-processing and Training

In the pre-processing step I will impute the missing values and from the data, drop redundent high correlation variables from the dataset, complete a stratified train test split and and investigate scaling the training data.  Because of the high amount of skewed data the possibility of transforming the data may need to be done after the initial models.  

## Import Statements

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from random import randint, choice
import random

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LinearRegression

## Load Data

In [2]:
#load bankruptcy data and column key data
bankruptcy_data = pd.read_csv('bankruptcy_data_comb.csv')
data_columns = pd.read_csv('column_key.csv')

#load high correlation dataframe
dataCorrhigh = pd.read_csv('dataCorrhigh.csv')

#display high correlation pairs dataframe
print(dataCorrhigh.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Feature1     111 non-null    object 
 1   Feature2     111 non-null    object 
 2   Correlation  111 non-null    float64
dtypes: float64(1), object(2)
memory usage: 2.7+ KB
None


## Impute missing data

Based of the distribution of data and the number of outliers, I have made the decsion to imput the missing values based on the median of each column.

In [3]:
#create new dataframe by imputing missing values with the median
bankruptcy_complete = bankruptcy_data.apply(lambda x: x.fillna(x.median()),axis=0)

#display complete dataframe
#bankruptcy_complete.info()

print(dataCorrhigh)

    Feature1 Feature2  Correlation
0        X07      X14     1.000000
1        X56      X58     0.999976
2        X04      X46     0.999920
3        X20      X58     0.999894
4        X20      X56     0.999880
..       ...      ...          ...
106      X07      X63     0.709315
107      X14      X63     0.709315
108      X44      X13     0.708622
109      X13      X19     0.706408
110      X63      X18     0.703221

[111 rows x 3 columns]


## Drop redundant high correlation  columns


In [4]:
# drop first row of correlation dataframe with correlation of 1.  Data is identical 
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh.index[0]).reset_index(drop=True)

#create empty final high correlation dataframe
corr_high_final = []
print(type(corr_high_final))

# remove redundant feature X07
dataCorrhigh = dataCorrhigh.drop(dataCorrhigh[(dataCorrhigh['Feature1'] == 'X07') | (dataCorrhigh['Feature2'] == 'X07')].index,axis = 0 ,inplace = False)
dataCorrhigh = dataCorrhigh.reset_index(drop=True)

# iterate over high correlation dataframe and use random number to determine final feture dataframe
for x in range(len(dataCorrhigh)):
    k = random.randint(0, 1)
    if k == 0:
        corr_high_final.append(dataCorrhigh['Feature1'][x])
        
    else:
        corr_high_final.append(dataCorrhigh['Feature2'][x])
   
print(corr_high_final)

<class 'list'>
['X56', 'X04', 'X20', 'X56', 'X08', 'X19', 'X31', 'X43', 'X10', 'X53', 'X31', 'X03', 'X46', 'X04', 'X14', 'X09', 'X56', 'X20', 'X58', 'X16', 'X56', 'X44', 'X44', 'X63', 'X22', 'X12', 'X48', 'X12', 'X49', 'X43', 'X49', 'X56', 'X49', 'X11', 'X63', 'X49', 'X32', 'X33', 'X32', 'X63', 'X36', 'X03', 'X51', 'X62', 'X64', 'X64', 'X44', 'X43', 'X22', 'X11', 'X33', 'X63', 'X63', 'X49', 'X33', 'X63', 'X14', 'X30', 'X30', 'X30', 'X48', 'X35', 'X18', 'X22', 'X22', 'X33', 'X11', 'X18', 'X62', 'X43', 'X33', 'X14', 'X62', 'X62', 'X62', 'X18', 'X10', 'X25', 'X02', 'X34', 'X36', 'X14', 'X18', 'X24', 'X24', 'X06', 'X06', 'X24', 'X24', 'X24', 'X34', 'X24', 'X13', 'X13', 'X20', 'X11', 'X34', 'X35', 'X43', 'X14', 'X44', 'X13', 'X63']


## Create final feature array 

In [5]:
# get unique feature array
feature_df = np.unique(corr_high_final)
print(feature_df)

['X02' 'X03' 'X04' 'X06' 'X08' 'X09' 'X10' 'X11' 'X12' 'X13' 'X14' 'X16'
 'X18' 'X19' 'X20' 'X22' 'X24' 'X25' 'X30' 'X31' 'X32' 'X33' 'X34' 'X35'
 'X36' 'X43' 'X44' 'X46' 'X48' 'X49' 'X51' 'X53' 'X56' 'X58' 'X62' 'X63'
 'X64']


## Remove non correlated columns from bankruptcy data

In [6]:
#create new dataframe with high correlation features
bankruptcy_data_final = bankruptcy_complete[feature_df].copy()

# add back in the year and class features
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Year']], axis = 1)
bankruptcy_data_final = pd.concat([bankruptcy_data_final,bankruptcy_complete['Class']], axis = 1)
print(bankruptcy_data_final.info()) # head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43405 entries, 0 to 43404
Data columns (total 39 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X02     43405 non-null  float64
 1   X03     43405 non-null  float64
 2   X04     43405 non-null  float64
 3   X06     43405 non-null  float64
 4   X08     43405 non-null  float64
 5   X09     43405 non-null  float64
 6   X10     43405 non-null  float64
 7   X11     43405 non-null  float64
 8   X12     43405 non-null  float64
 9   X13     43405 non-null  float64
 10  X14     43405 non-null  float64
 11  X16     43405 non-null  float64
 12  X18     43405 non-null  float64
 13  X19     43405 non-null  float64
 14  X20     43405 non-null  float64
 15  X22     43405 non-null  float64
 16  X24     43405 non-null  float64
 17  X25     43405 non-null  float64
 18  X30     43405 non-null  float64
 19  X31     43405 non-null  float64
 20  X32     43405 non-null  float64
 21  X33     43405 non-null  float64
 22

## Drop column variables from column key dataframe

## Train test split


In [7]:
X_train, X_test, y_train, y_test = train_test_split(bankruptcy_data_final.drop(columns='Class'), bankruptcy_data_final.Class,
                                                    test_size=.25  ,random_state=5, stratify=bankruptcy_data_final.Class)

print(X_train)

           X02      X03      X04       X06      X08      X09      X10  \
12113  1.39110 -0.83283  0.39990 -1.630900 -0.31339  0.88906 -0.43594   
3289   0.25636  0.27021  2.05400  0.035034  2.84740  0.98773  0.72997   
16567  0.54284 -0.12099  0.72904  0.188430  0.74268  1.00990  0.40316   
21367  0.54860  0.37752  2.66550  0.297050  0.78826  1.12700  0.43244   
32394  0.10553  0.60189  6.70350  0.390000  8.44050  1.16000  0.89073   
...        ...      ...      ...       ...      ...      ...      ...   
24299  2.15570 -0.69448  0.25470 -0.072920 -0.53611  1.14410 -1.15570   
41350  0.43133  0.28017  1.64960  0.217130  1.25060  1.07930  0.53940   
38703  0.55160  0.17282  1.91770  0.000000  0.81292  1.18680  0.44840   
12352  0.15038  0.49805  5.54020 -0.000078  5.64980  0.99346  0.84962   
24962  0.69258 -0.24260  0.54536 -0.054472  0.44388  6.12300  0.30742   

            X11       X12       X13  ...       X48       X49      X51  \
12113 -1.068200 -0.769670 -0.104620  ... -1.855800

In [8]:
X_train.shape, X_test.shape

((32553, 38), (10852, 38))

In [9]:
y_train.shape, y_test.shape

((32553,), (10852,))

In [10]:
print(X_train.columns)

Index(['X02', 'X03', 'X04', 'X06', 'X08', 'X09', 'X10', 'X11', 'X12', 'X13',
       'X14', 'X16', 'X18', 'X19', 'X20', 'X22', 'X24', 'X25', 'X30', 'X31',
       'X32', 'X33', 'X34', 'X35', 'X36', 'X43', 'X44', 'X46', 'X48', 'X49',
       'X51', 'X53', 'X56', 'X58', 'X62', 'X63', 'X64', 'Year'],
      dtype='object')


In [11]:
#drop year column

In [12]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
clf = LogisticRegression(max_iter=1000)

clf.fit(X_train_scaled, y_train) 


LogisticRegression(max_iter=1000)

In [14]:
#y_tr_pred = clf.predict(X_train_scaled)
y_te_pred = clf.predict(X_test_scaled)

In [16]:
print("\n")
print("[Test] Accuracy score (y_predict_test, ytestlr):",accuracy_score(y_te_pred, y_test))

print("\n")
print("[Test] Accuracy score: (y_test, y_te_pred)",accuracy_score(y_test, y_te_pred))

y_predict_training = clf.predict(X_train_scaled)
print("\n")
print("[Training] Accuracy score: (y_train, y_predict_training)",accuracy_score(y_train, y_predict_training))






[Test] Accuracy score (y_predict_test, ytestlr): 0.9517139697751567


[Test] Accuracy score: (y_test, y_te_pred) 0.9517139697751567


[Training] Accuracy score: (y_train, y_predict_training) 0.9518323963997174
